#### Warming stripes - final code 

- Wetterstationen_liste.csv
- https://dataset.api.hub.geosphere.at/v1/openapi-docs#/station/Historical_Station_Data_station_historical__resource_id__get 
    - eingeben: https://dataset.api.hub.geosphere.at/v1/station/historical/klima-v2-1m?parameters=tl_mittel&start=1800-01-01T00%3A00&end=2023-12-01T00%3A00&station_ids=1"
    - station_ids = finden wir hier: https://dataset.api.hub.geosphere.at/v1/station/historical/klima-v2-1m/metadata 

    

#### höchstgelegene Wetterstation: Sonnblick 
https://dataset.api.hub.geosphere.at/v1/station/historical/klima-v2-1m?parameters=tl_mittel&start=1768-01-01&end=2023-12-31&station_ids=213&output_format=csv&filename=sonnblick"

In [3]:
# pandas importieren
import pandas as pd

# requests importieren
import requests  

# numpy importieren 
import numpy as np

# matplot importieren
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
import matplotlib.cm as cm

# ich möchte alle Reihen und Spalten sehen, wenn ich will
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)



### STEP 1: wir nutzen den fast API Swagger um eine Tabelle downzuloaden 

### Quelle: https://dataset.api.hub.geosphere.at/v1/openapi-docs#/station/Historical_Station_Data_station_historical__resource_id__get

- höchstgelegene Wetterstation: Sonnblick 
- https://dataset.api.hub.geosphere.at/v1/station/historical/klima-v2-1m?parameters=tl_mittel&start=1768-01-01&end=2023-12-31&station_ids=213&output_format=csv&filename=sonnblick"

In [13]:
# Daten einlesen und ich möchte die ersten 5 Ergebnisse sehen 
data = pd.read_csv("sonnblick.csv")
data.head()

,time,station,tl_mittel,substation
0,1768-01-01T00:00+00:00,213,NaN,NaN
1,1768-02-01T00:00+00:00,213,NaN,NaN
2,1768-03-01T00:00+00:00,213,NaN,NaN
3,1768-04-01T00:00+00:00,213,NaN,NaN
4,1768-05-01T00:00+00:00,213,NaN,NaN


In [15]:
# ich mach mir aus der Time-Spalte ein datetime-Format 
data['time'] = pd.to_datetime(data['time'])
data.info() 


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3072 entries, 0 to 3071
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype              
---  ------      --------------  -----              
 0   time        3072 non-null   datetime64[ns, UTC]
 1   station     3072 non-null   int64              
 2   tl_mittel   1647 non-null   float64            
 3   substation  1647 non-null   float64            
dtypes: datetime64[ns, UTC](1), float64(2), int64(1)
memory usage: 96.1 KB


In [16]:
# Aufzeichnungen gibts ab 1887 (erstes volles Jahr der Aufzeichnungen)
# dt.year ist der Befehl bei datetime, um nur das Jahr zu bekommen
data = data[data['time'].dt.year >= 1887]

print(data.head())

                          time  station  tl_mittel  substation
1428 1887-01-01 00:00:00+00:00      213      -12.6     15410.0
1429 1887-02-01 00:00:00+00:00      213      -15.7     15410.0
1430 1887-03-01 00:00:00+00:00      213      -10.7     15410.0
1431 1887-04-01 00:00:00+00:00      213       -9.7     15410.0
1432 1887-05-01 00:00:00+00:00      213       -6.7     15410.0


In [17]:
#Ich hätte gerne nur zwei Spalten, nämlich time und die mittlere Temperatur 
data = data[['time', 'tl_mittel']]
print(data.head()) 

                          time  tl_mittel
1428 1887-01-01 00:00:00+00:00      -12.6
1429 1887-02-01 00:00:00+00:00      -15.7
1430 1887-03-01 00:00:00+00:00      -10.7
1431 1887-04-01 00:00:00+00:00       -9.7
1432 1887-05-01 00:00:00+00:00       -6.7


In [18]:
# jetzt hätte ich gerne für jedes Jahr den Mittelwert (also einen Wert aus den 12 Monatsangeben)
# wenn keine Angabe (NaN), dann wird dieses eine Element nicht berücksichtigt

# ich mache eine neue Spalte nur mit year und hol mir die aus der Spalte "time" raus  
data['year'] = data['time'].dt.year
data.head()


,time,tl_mittel,year
1428,1887-01-01 00:00:00+00:00,-12.6,1887
1429,1887-02-01 00:00:00+00:00,-15.7,1887
1430,1887-03-01 00:00:00+00:00,-10.7,1887
1431,1887-04-01 00:00:00+00:00,-9.7,1887
1432,1887-05-01 00:00:00+00:00,-6.7,1887


In [19]:
# jetzt gruppiere ich die Monatsangaben pro Jahr und nehme den Mittelwert
df = data.groupby('year')['tl_mittel'].mean().reset_index()

# ich mach mir zwei Spalten
df.columns = ['year', 'mean_temperature']

df.head()


,year,mean_temperature
0,1887,-7.466667
1,1888,-6.808333
2,1889,-7.125000
3,1890,-6.983333
4,1891,-6.733333


In [20]:
# wir konvertieren unser Jahr wieder in ein datetime-Format
df['year'] = pd.to_datetime(df['year'], format='%Y')
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137 entries, 0 to 136
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   year              137 non-null    datetime64[ns]
 1   mean_temperature  137 non-null    float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 2.3 KB


,year,mean_temperature
0,1887-01-01,-7.466667
1,1888-01-01,-6.808333
2,1889-01-01,-7.125000
3,1890-01-01,-6.983333
4,1891-01-01,-6.733333


In [21]:
# ich hätte gerne als Ausgangspunkt (baseline) den Mittelwert der Jahre 1961 - 2010 
baseline_period = df[(df['year'].dt.year >= 1961) & (df['year'].dt.year <= 2010)]
baseline_avg = baseline_period['mean_temperature'].mean()

# Calculate standard deviation (1901-2000)
std_period = df[(df['year'].dt.year >= 1901) & (df['year'].dt.year <= 2000)]
std_dev = std_period['mean_temperature'].std()

# und dann rechne ich mir die jährliche Anomalie aus 
df['annual_anomaly'] = (df['mean_temperature'] - baseline_avg) / std_dev

# Das neue dataframe heißt results und hat zwei Spalten: year und annual_anomaly
result = df[['year', 'annual_anomaly']]
print(result.head())

        year  annual_anomaly
0 1887-01-01       -2.748785
1 1888-01-01       -1.850861
2 1889-01-01       -2.282774
3 1890-01-01       -2.089550
4 1891-01-01       -1.748566


In [22]:
# und jetzt möchte ich nur das jahr haben 
result['year'] = result['year'].dt.strftime('%Y')

# und die jährliche Anomalie auf 3 Kommastellen gerundet
result['annual_anomaly'] = result['annual_anomaly'].round(3)

print(result.head())

   year  annual_anomaly
0  1887          -2.749
1  1888          -1.851
2  1889          -2.283
3  1890          -2.090
4  1891          -1.749


C:\Users\linde\AppData\Local\Temp\ipykernel_23296\1504311656.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['year'] = result['year'].dt.strftime('%Y')
C:\Users\linde\AppData\Local\Temp\ipykernel_23296\1504311656.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['annual_anomaly'] = result['annual_anomaly'].round(3)


In [25]:
# jetzt hätte ich gerne ein excel
result.to_excel("sonnblick.xlsx", index=False)

In [26]:
# im excel schreibe ich in die letzte Zeile folgendes: &nbsp;	<br/>


In [ ]:
# Rest in Datawrapper

#### tiegstgelegene Wetterstation: Neusiedl am See 
https://dataset.api.hub.geosphere.at/v1/station/historical/klima-v2-1m?parameters=tl_mittel&start=1768-01-01&end=2023-12-31&station_ids=65&output_format=csv&filename=neusiedl/see"

In [29]:
# Daten einlesen und ich möchte die ersten 5 Ergebnisse sehen 
data = pd.read_csv("neusiedl_see.csv")
data.head()

,time,station,tl_mittel,substation
0,1768-01-01T00:00+00:00,65,NaN,NaN
1,1768-02-01T00:00+00:00,65,NaN,NaN
2,1768-03-01T00:00+00:00,65,NaN,NaN
3,1768-04-01T00:00+00:00,65,NaN,NaN
4,1768-05-01T00:00+00:00,65,NaN,NaN


In [30]:
# ich mach mir aus der Time-Spalte ein datetime-Format 
data['time'] = pd.to_datetime(data['time'])
data.info()

# Aufzeichnungen gibts ab 1939 (erstes volles Jahr der Aufzeichnungen)
# dt.year ist der Befehl bei datetime, um nur das Jahr zu bekommen
data = data[data['time'].dt.year >= 1939]

print(data.head())

#Ich hätte gerne nur zwei Spalten, nämlich time und die mittlere Temperatur 
data = data[['time', 'tl_mittel']]
print(data.head()) 

# jetzt hätte ich gerne für jedes Jahr den Mittelwert (also einen Wert aus den 12 Monatsangeben)
# wenn keine Angabe (NaN), dann wird dieses eine Element nicht berücksichtigt

# ich mache eine neue Spalte nur mit year und hol mir die aus der Spalte "time" raus  
data['year'] = data['time'].dt.year
data.head()

# jetzt gruppiere ich die Monatsangaben pro Jahr und nehme den Mittelwert
df = data.groupby('year')['tl_mittel'].mean().reset_index()

# ich mach mir zwei Spalten
df.columns = ['year', 'mean_temperature']

df.head()

# wir konvertieren unser Jahr wieder in ein datetime-Format
df['year'] = pd.to_datetime(df['year'], format='%Y')
df.info()
df.head()

# ich hätte gerne als Ausgangspunkt (baseline) den Mittelwert der Jahre 1961 - 2010 
baseline_period = df[(df['year'].dt.year >= 1961) & (df['year'].dt.year <= 2010)]
baseline_avg = baseline_period['mean_temperature'].mean()

# Calculate standard deviation (1901-2000)
std_period = df[(df['year'].dt.year >= 1901) & (df['year'].dt.year <= 2000)]
std_dev = std_period['mean_temperature'].std()

# und dann rechne ich mir die jährliche Anomalie aus 
df['annual_anomaly'] = (df['mean_temperature'] - baseline_avg) / std_dev

# Das neue dataframe heißt results und hat zwei Spalten: year und annual_anomaly
result = df[['year', 'annual_anomaly']]
print(result.head())

# und jetzt möchte ich nur das jahr haben 
result['year'] = result['year'].dt.strftime('%Y')

# und die jährliche Anomalie auf 3 Kommastellen gerundet
result['annual_anomaly'] = result['annual_anomaly'].round(3)

print(result.head())

# jetzt hätte ich gerne ein excel
result.to_excel("neusiedl_see.xlsx", index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3072 entries, 0 to 3071
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype              
---  ------      --------------  -----              
 0   time        3072 non-null   datetime64[ns, UTC]
 1   station     3072 non-null   int64              
 2   tl_mittel   969 non-null    float64            
 3   substation  1010 non-null   float64            
dtypes: datetime64[ns, UTC](1), float64(2), int64(1)
memory usage: 96.1 KB
                          time  station  tl_mittel  substation
2052 1939-01-01 00:00:00+00:00       65        0.6      7901.0
2053 1939-02-01 00:00:00+00:00       65        2.4      7901.0
2054 1939-03-01 00:00:00+00:00       65        2.5      7901.0
2055 1939-04-01 00:00:00+00:00       65       13.1      7901.0
2056 1939-05-01 00:00:00+00:00       65       13.7      7901.0
                          time  tl_mittel
2052 1939-01-01 00:00:00+00:00        0.6
2053 1939-02-01 00:00:00+00:00        2

C:\Users\linde\AppData\Local\Temp\ipykernel_23296\2442220910.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['year'] = result['year'].dt.strftime('%Y')
C:\Users\linde\AppData\Local\Temp\ipykernel_23296\2442220910.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['annual_anomaly'] = result['annual_anomaly'].round(3)


#### Wien - Hohe Warte
https://dataset.api.hub.geosphere.at/v1/station/historical/klima-v2-1m?parameters=tl_mittel&start=1768-01-01&end=2023-12-31&station_ids=105&output_format=csv&filename=hohe_warte"

In [35]:
# Daten einlesen und ich möchte die ersten 5 Ergebnisse sehen 
data = pd.read_csv("hohe_warte.csv")
data.head()

,time,station,tl_mittel,substation
0,1768-01-01T00:00+00:00,105,NaN,NaN
1,1768-02-01T00:00+00:00,105,NaN,NaN
2,1768-03-01T00:00+00:00,105,NaN,NaN
3,1768-04-01T00:00+00:00,105,NaN,NaN
4,1768-05-01T00:00+00:00,105,NaN,NaN


In [36]:
# ich mach mir aus der Time-Spalte ein datetime-Format 
data['time'] = pd.to_datetime(data['time'])
data.info()

# Aufzeichnungen gibts ab 1775 (erstes volles Jahr der Aufzeichnungen)
# dt.year ist der Befehl bei datetime, um nur das Jahr zu bekommen
data = data[data['time'].dt.year >= 1775]

print(data.head())

#Ich hätte gerne nur zwei Spalten, nämlich time und die mittlere Temperatur 
data = data[['time', 'tl_mittel']]
print(data.head()) 

# jetzt hätte ich gerne für jedes Jahr den Mittelwert (also einen Wert aus den 12 Monatsangeben)
# wenn keine Angabe (NaN), dann wird dieses eine Element nicht berücksichtigt

# ich mache eine neue Spalte nur mit year und hol mir die aus der Spalte "time" raus  
data['year'] = data['time'].dt.year
data.head()

# jetzt gruppiere ich die Monatsangaben pro Jahr und nehme den Mittelwert
df = data.groupby('year')['tl_mittel'].mean().reset_index()

# ich mach mir zwei Spalten
df.columns = ['year', 'mean_temperature']

df.head()

# wir konvertieren unser Jahr wieder in ein datetime-Format
df['year'] = pd.to_datetime(df['year'], format='%Y')
df.info()
df.head()

# ich hätte gerne als Ausgangspunkt (baseline) den Mittelwert der Jahre 1961 - 2010 
baseline_period = df[(df['year'].dt.year >= 1961) & (df['year'].dt.year <= 2010)]
baseline_avg = baseline_period['mean_temperature'].mean()

# Calculate standard deviation (1901-2000)
std_period = df[(df['year'].dt.year >= 1901) & (df['year'].dt.year <= 2000)]
std_dev = std_period['mean_temperature'].std()

# und dann rechne ich mir die jährliche Anomalie aus 
df['annual_anomaly'] = (df['mean_temperature'] - baseline_avg) / std_dev

# Das neue dataframe heißt results und hat zwei Spalten: year und annual_anomaly
result = df[['year', 'annual_anomaly']]
print(result.head())

# und jetzt möchte ich nur das jahr haben 
result['year'] = result['year'].dt.strftime('%Y')

# und die jährliche Anomalie auf 3 Kommastellen gerundet
result['annual_anomaly'] = result['annual_anomaly'].round(3)

print(result.head())

# jetzt hätte ich gerne ein excel
result.to_excel("hohe_warte.xlsx", index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3072 entries, 0 to 3071
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype              
---  ------      --------------  -----              
 0   time        3072 non-null   datetime64[ns, UTC]
 1   station     3072 non-null   int64              
 2   tl_mittel   2983 non-null   float64            
 3   substation  2986 non-null   float64            
dtypes: datetime64[ns, UTC](1), float64(2), int64(1)
memory usage: 96.1 KB
                        time  station  tl_mittel  substation
84 1775-01-01 00:00:00+00:00      105       -2.6      5905.0
85 1775-02-01 00:00:00+00:00      105        2.3      5905.0
86 1775-03-01 00:00:00+00:00      105        4.7      5905.0
87 1775-04-01 00:00:00+00:00      105        6.2      5905.0
88 1775-05-01 00:00:00+00:00      105       10.2      5905.0
                        time  tl_mittel
84 1775-01-01 00:00:00+00:00       -2.6
85 1775-02-01 00:00:00+00:00        2.3
86 1775-03-01 0

C:\Users\linde\AppData\Local\Temp\ipykernel_23296\1578270888.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['year'] = result['year'].dt.strftime('%Y')
C:\Users\linde\AppData\Local\Temp\ipykernel_23296\1578270888.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['annual_anomaly'] = result['annual_anomaly'].round(3)


#### OÖ - Kremsmünster
https://dataset.api.hub.geosphere.at/v1/station/historical/klima-v2-1m?parameters=tl_mittel&start=1768-01-01&end=2023-12-31&station_ids=204&output_format=csv&filename=kremsmunster"

- hier haben wir zwei Datensätze - einer bis 2008 (kremsmunster_1) und einer von 2008 - 2023 (kremsmunster_2)
- wir haben sie beiden aifbereitet und dann in excel kremsmunster_1 zusammengefügt 8 = aktuelle Liste

In [42]:
# Daten einlesen und ich möchte die ersten 5 Ergebnisse sehen 
data = pd.read_csv("kremsmunster_1.csv")
data.head()

# ich mach mir aus der Time-Spalte ein datetime-Format 
data['time'] = pd.to_datetime(data['time'])
data.info()

# Aufzeichnungen gibts ab 1768 (erstes volles Jahr der Aufzeichnungen)
# dt.year ist der Befehl bei datetime, um nur das Jahr zu bekommen
data = data[data['time'].dt.year >= 1768]

print(data.head())

#Ich hätte gerne nur zwei Spalten, nämlich time und die mittlere Temperatur 
data = data[['time', 'tl_mittel']]
print(data.head()) 

# jetzt hätte ich gerne für jedes Jahr den Mittelwert (also einen Wert aus den 12 Monatsangeben)
# wenn keine Angabe (NaN), dann wird dieses eine Element nicht berücksichtigt

# ich mache eine neue Spalte nur mit year und hol mir die aus der Spalte "time" raus  
data['year'] = data['time'].dt.year
data.head()

# jetzt gruppiere ich die Monatsangaben pro Jahr und nehme den Mittelwert
df = data.groupby('year')['tl_mittel'].mean().reset_index()

# ich mach mir zwei Spalten
df.columns = ['year', 'mean_temperature']

df.head()

# wir konvertieren unser Jahr wieder in ein datetime-Format
df['year'] = pd.to_datetime(df['year'], format='%Y')
df.info()
df.head()

# ich hätte gerne als Ausgangspunkt (baseline) den Mittelwert der Jahre 1961 - 2010 
baseline_period = df[(df['year'].dt.year >= 1961) & (df['year'].dt.year <= 2010)]
baseline_avg = baseline_period['mean_temperature'].mean()

# Calculate standard deviation (1901-2000)
std_period = df[(df['year'].dt.year >= 1901) & (df['year'].dt.year <= 2000)]
std_dev = std_period['mean_temperature'].std()

# und dann rechne ich mir die jährliche Anomalie aus 
df['annual_anomaly'] = (df['mean_temperature'] - baseline_avg) / std_dev

# Das neue dataframe heißt results und hat zwei Spalten: year und annual_anomaly
result = df[['year', 'annual_anomaly']]
print(result.head())

# und jetzt möchte ich nur das jahr haben 
result['year'] = result['year'].dt.strftime('%Y')

# und die jährliche Anomalie auf 3 Kommastellen gerundet
result['annual_anomaly'] = result['annual_anomaly'].round(3)

print(result.head())

# jetzt hätte ich gerne ein excel
result.to_excel("kremsmunster_n.xlsx", index=False)



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3072 entries, 0 to 3071
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype              
---  ------     --------------  -----              
 0   time       3072 non-null   datetime64[ns, UTC]
 1   station    3072 non-null   int64              
 2   tl_mittel  2891 non-null   float64            
dtypes: datetime64[ns, UTC](1), float64(1), int64(1)
memory usage: 72.1 KB
                       time  station  tl_mittel
0 1768-01-01 00:00:00+00:00     5010       -9.4
1 1768-02-01 00:00:00+00:00     5010       -4.8
2 1768-03-01 00:00:00+00:00     5010        8.6
3 1768-04-01 00:00:00+00:00     5010        7.7
4 1768-05-01 00:00:00+00:00     5010       14.0
                       time  tl_mittel
0 1768-01-01 00:00:00+00:00       -9.4
1 1768-02-01 00:00:00+00:00       -4.8
2 1768-03-01 00:00:00+00:00        8.6
3 1768-04-01 00:00:00+00:00        7.7
4 1768-05-01 00:00:00+00:00       14.0
<class 'pandas.core.frame.DataFrame'>

C:\Users\linde\AppData\Local\Temp\ipykernel_23296\255391643.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['year'] = result['year'].dt.strftime('%Y')
C:\Users\linde\AppData\Local\Temp\ipykernel_23296\255391643.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['annual_anomaly'] = result['annual_anomaly'].round(3)


In [43]:
# Daten einlesen und ich möchte die ersten 5 Ergebnisse sehen 
data = pd.read_csv("kremsmunster_2.csv")
data.head()

# ich mach mir aus der Time-Spalte ein datetime-Format 
data['time'] = pd.to_datetime(data['time'])
data.info()

# Aufzeichnungen gibts ab 1768 (erstes volles Jahr der Aufzeichnungen)
# dt.year ist der Befehl bei datetime, um nur das Jahr zu bekommen
data = data[data['time'].dt.year >= 1768]

print(data.head())

#Ich hätte gerne nur zwei Spalten, nämlich time und die mittlere Temperatur 
data = data[['time', 'tl_mittel']]
print(data.head()) 

# jetzt hätte ich gerne für jedes Jahr den Mittelwert (also einen Wert aus den 12 Monatsangeben)
# wenn keine Angabe (NaN), dann wird dieses eine Element nicht berücksichtigt

# ich mache eine neue Spalte nur mit year und hol mir die aus der Spalte "time" raus  
data['year'] = data['time'].dt.year
data.head()

# jetzt gruppiere ich die Monatsangaben pro Jahr und nehme den Mittelwert
df = data.groupby('year')['tl_mittel'].mean().reset_index()

# ich mach mir zwei Spalten
df.columns = ['year', 'mean_temperature']

df.head()

# wir konvertieren unser Jahr wieder in ein datetime-Format
df['year'] = pd.to_datetime(df['year'], format='%Y')
df.info()
df.head()

# ich hätte gerne als Ausgangspunkt (baseline) den Mittelwert der Jahre 1961 - 2010 
baseline_period = df[(df['year'].dt.year >= 1961) & (df['year'].dt.year <= 2010)]
baseline_avg = baseline_period['mean_temperature'].mean()

# Calculate standard deviation (1901-2000)
std_period = df[(df['year'].dt.year >= 1901) & (df['year'].dt.year <= 2000)]
std_dev = std_period['mean_temperature'].std()

# und dann rechne ich mir die jährliche Anomalie aus 
df['annual_anomaly'] = (df['mean_temperature'] - baseline_avg) / std_dev

# Das neue dataframe heißt results und hat zwei Spalten: year und annual_anomaly
result = df[['year', 'annual_anomaly']]
print(result.head())

# und jetzt möchte ich nur das jahr haben 
result['year'] = result['year'].dt.strftime('%Y')

# und die jährliche Anomalie auf 3 Kommastellen gerundet
result['annual_anomaly'] = result['annual_anomaly'].round(3)

print(result.head())

# jetzt hätte ich gerne ein excel
result.to_excel("kremsmunster.xlsx", index=False)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3072 entries, 0 to 3071
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype              
---  ------      --------------  -----              
 0   time        3072 non-null   datetime64[ns, UTC]
 1   station     3072 non-null   int64              
 2   tl_mittel   1799 non-null   float64            
 3   substation  1799 non-null   float64            
dtypes: datetime64[ns, UTC](1), float64(2), int64(1)
memory usage: 96.1 KB
                       time  station  tl_mittel  substation
0 1768-01-01 00:00:00+00:00      204        NaN         NaN
1 1768-02-01 00:00:00+00:00      204        NaN         NaN
2 1768-03-01 00:00:00+00:00      204        NaN         NaN
3 1768-04-01 00:00:00+00:00      204        NaN         NaN
4 1768-05-01 00:00:00+00:00      204        NaN         NaN
                       time  tl_mittel
0 1768-01-01 00:00:00+00:00        NaN
1 1768-02-01 00:00:00+00:00        NaN
2 1768-03-01 00:00:00+00

C:\Users\linde\AppData\Local\Temp\ipykernel_23296\4183401692.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['year'] = result['year'].dt.strftime('%Y')
C:\Users\linde\AppData\Local\Temp\ipykernel_23296\4183401692.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['annual_anomaly'] = result['annual_anomaly'].round(3)
